In [1]:
from sequential_tasks import EchoData
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
class TimeDistributed(nn.Module):
    def __init__(self, module):
        """
        Collapses input of dim T*N*H to (T*N)*H, and applies to a module.
        Allows handling of variable sequence lengths and minibatch sizes.
        :param module: Module to apply input to.
        """
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        t, n = x.size(0), x.size(1)
        x = x.view(t * n, -1)
        x = self.module(x)
        x = x.view(t, n, -1)
        return x

    def __repr__(self):
        tmpstr = self.__class__.__name__ + ' (\n'
        tmpstr += self.module.__repr__()
        tmpstr += ')'
        return tmpstr

In [4]:
class SimpleRNN(nn.Module):

    def __init__(self, input_size, rnn_hidden_size, output_size, maxlen):

        super(SimpleRNN, self).__init__()
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = torch.nn.RNN(input_size, self.rnn_hidden_size, num_layers=1, nonlinearity='relu', batch_first=True)
        self.linear = TimeDistributed(torch.nn.Linear(rnn_hidden_size, 1))
        self.maxlen = maxlen

    def forward(self, x, hidden):
        x, hidden = self.rnn(x, hidden)
        #x = x.select(1, 4).contiguous()
        #x = x.view(-1, self.rnn_hidden_size)        
        x = nn.Sigmoid()(self.linear(hidden).view(-1))
        return x, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return weight.new(1, batch_size, self.rnn_hidden_size).zero_()

In [5]:
def echo_experiment(settings):
    train_data_gen = EchoData(
        series_length=settings['series_length'],
        truncated_length=settings['truncated_length'],
        echo_step=settings['echo_step'],
        batch_size=settings['batch_size'])

    feature_dim = 1 #since we have a scalar series
    model = SimpleRNN(feature_dim, settings['h_units'], 1, train_data_gen.series_length)  

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

    epochs = settings['max_epochs']
    log_interval = settings['log_interval']

    epoch = 0
    hidden = model.init_hidden(train_data_gen.batch_size)
    while epoch < epochs:
        predictions = []
        truth_values = []
   
        for batch_idx, (xs, ys) in enumerate(train_data_gen):
            xs, ys = torch.from_numpy(xs).float(), torch.from_numpy(ys).long()

            y_pred, hidden = model(xs, hidden)
            loss = criterion(y_pred, ys)
            optimizer.zero_grad()
            loss.backward()
            #nn.utils.clip_grad_norm(model.parameters(), 0.5)
            optimizer.step()

            predictions.append(y_pred.cpu().data.numpy().ravel())
            truth_values.append(ys.cpu().data.numpy().ravel())

            if batch_idx % log_interval == 0:
                print('Train Epoch: {}, mini-batch {} of {}, training loss: {:.6f}'.format(
                    epoch, batch_idx, len(train_data_gen), loss.item()))

        epoch += 1

    # testing
    test_data_gen = EchoData(
        series_length=settings['series_length'],
        truncated_length=settings['truncated_length'],
        echo_step=settings['echo_step'],
        batch_size=settings['batch_size'])

    correct = 0
    #with torch.no_grad():
    #    for i,(xs,ys) in enumerate(test_data_gen):
    #        xs, ys = torch.from_numpy(xs).float(), torch.from_numpy(ys).long()
    #        output = model(xs)
    #        pred = output.max(1, keepdim=True)[1]
    #        correct += pred.eq(ys.view_as(pred)).sum().item()

    test_accuracy = float(correct) / len(test_data_gen)
    print('\nAccuracy: {}'.format(test_accuracy))

    return test_accuracy

In [10]:
# experiments settings
params = {
    "series_length": 20000,
    "echo_step": 3,
    "truncated_length": 20,
    "batch_size": 5,
    "h_units": 4,
    "max_epochs": 4,
    "log_interval": 10
}

acc = echo_experiment(params)
print('acc = {:.2f}%.'.format(acc))

RuntimeError: select(): index 19999 out of range for tensor of size [5, 20, 4] at dimension 1